In [1]:
import os
cwd = os.getcwd().replace('/vectorbt', '')
os.chdir(cwd)

In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import pandas_ta as ta
vbt.settings.set_theme('dark')


In [3]:
dataset = pd.read_csv('data/dataset_5m_180day_tickers.csv' , index_col=0)
dataset.index = pd.to_datetime(dataset.index)

In [5]:
data = dataset.copy().\
    groupby('symbol')['close']\
    .resample('30Min')\
    .last()\
    .unstack()\
    .T
data


symbol,ADAUSDT,ALGOUSDT,ATOMUSDT,BCHUSDT,BNBUSDT,BTCUSDT,DOGEUSDT,DOTUSDT,ETCUSDT,ETHUSDT,FILUSDT,LINKUSDT,LTCUSDT,LUNAUSDT,MATICUSDT,SOLUSDT,XLMUSDT,XRPUSDT
time,,,,,,,,,,,,,,,,,,
2021-11-01 00:00:00,1.9710,1.8528,37.72,597.8,527.7,61468.98,0.2762,43.27,54.70,4311.62,63.74,30.26,193.2,43.12,1.925,206.87,0.3733,1.1169
2021-11-01 00:30:00,1.9700,1.8511,37.49,598.4,528.2,61494.80,0.2826,43.23,54.55,4314.24,63.73,30.27,193.2,43.21,1.958,207.90,0.3739,1.1290
2021-11-01 01:00:00,1.9650,1.8352,36.89,595.9,528.1,61641.07,0.2785,43.32,54.41,4309.74,63.10,30.32,193.0,43.16,1.953,206.35,0.3722,1.1170
2021-11-01 01:30:00,1.9580,1.8331,36.59,594.4,526.5,61389.87,0.2732,43.13,54.09,4297.10,63.13,30.17,192.2,43.04,1.926,205.98,0.3692,1.1093
2021-11-01 02:00:00,1.9310,1.8083,35.73,584.4,518.0,60225.00,0.2681,42.51,53.06,4220.42,62.16,29.51,189.1,42.08,1.886,200.47,0.3641,1.0909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30 09:30:00,0.8103,0.6342,19.02,294.7,396.8,38613.53,0.1352,16.14,29.19,2831.33,17.38,12.01,100.6,83.87,1.142,94.01,0.1809,0.6216
2022-04-30 10:00:00,0.8070,0.6321,18.98,294.4,397.1,38589.13,0.1351,16.11,28.99,2831.24,17.32,11.99,100.5,83.65,1.137,93.99,0.1804,0.6195
2022-04-30 10:30:00,0.8059,0.6331,18.94,293.7,396.8,38572.83,0.1349,16.02,28.96,2830.78,17.35,12.03,100.4,84.05,1.138,93.91,0.1806,0.6203


#### Testing Moving Aeverage

In [6]:
# fast_ma = vbt.MA.run(data, [10, 20, 60, 120], short_name='fast')
# slow_ma = vbt.MA.run(data, [30, 30, 120, 240], short_name='slow')
grid = np.arange(10, 250, 10)
fast, slow = vbt.MA.run_combs(
    data,
    window=grid,
    r=2,
    short_names=['fast', 'low'])

entries = fast.ma_crossed_above(slow)
exits = fast.ma_crossed_below(slow)


In [7]:
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
comb = pf.total_return().unstack('symbol')
comb

symbol                   ADAUSDT  ALGOUSDT  ATOMUSDT   BCHUSDT   BNBUSDT  \
fast_window low_window                                                     
10          20         -0.589982 -0.698390 -0.490278 -0.540794 -0.399395   
            30         -0.456453 -0.681878 -0.228293 -0.619755 -0.283317   
            40         -0.490495 -0.647188 -0.080610 -0.603280 -0.163943   
            50         -0.349432 -0.671409  0.027586 -0.550233 -0.216175   
            60         -0.361545 -0.681578 -0.326147 -0.548899 -0.423069   
...                          ...       ...       ...       ...       ...   
210         230        -0.358484 -0.446444 -0.371292 -0.289247 -0.485267   
            240        -0.333709 -0.541932 -0.149479 -0.310036 -0.491759   
220         230        -0.487867 -0.631082 -0.321667 -0.340571 -0.528675   
            240        -0.379150 -0.619056 -0.174153 -0.344963 -0.488525   
230         240        -0.430742 -0.639829 -0.397028 -0.474641 -0.483560   

symbol                   BTCUSDT  DOGEUSDT   DOTUSDT   ETCUSDT   ETHUSDT  \
fast_window low_window                                                     
10          20         -0.576055 -0.564220 -0.631873 -0.452152 -0.366956   
            30         -0.439580 -0.559415 -0.680112 -0.352434 -0.341760   
            40         -0.352400 -0.400522 -0.591849 -0.442114 -0.258024   
            50         -0.412195 -0.359201 -0.620756 -0.406317 -0.143619   
            60         -0.436348 -0.417215 -0.500136 -0.434378 -0.278081   
...                          ...       ...       ...       ...       ...   
210         230        -0.151673 -0.508294 -0.437625 -0.067328 -0.352987   
            240        -0.214609 -0.581222 -0.465857  0.016286 -0.409279   
220         230        -0.279795 -0.601250 -0.564399 -0.220049 -0.460136   
            240        -0.316881 -0.575520 -0.516121  0.042843 -0.489198   
230         240        -0.343996 -0.528158 -0.611179 -0.107784 -0.506679   

symbol                   FILUSDT  LINKUSDT   LTCUSDT  LUNAUSDT  MATICUSDT  \
fast_window low_window                                                      
10          20         -0.579377 -0.447046 -0.464239 -0.204469  -0.632509   
            30         -0.560551 -0.603033 -0.509186 -0.189261  -0.719792   
            40         -0.550229 -0.694822 -0.580302 -0.298484  -0.567455   
            50         -0.493583 -0.692060 -0.545061  0.039762  -0.513729   
            60         -0.436347 -0.620359 -0.326805 -0.277335  -0.549585   
...                          ...       ...       ...       ...        ...   
210         230        -0.133205 -0.348205 -0.324822  0.801347  -0.252497   
            240        -0.013587 -0.407604 -0.405854  0.588226  -0.312487   
220         230        -0.132549 -0.498586 -0.383812  0.365145  -0.206142   
            240        -0.130032 -0.495630 -0.408146  0.231331  -0.302569   
230         240        -0.338050 -0.469959 -0.520855  0.195520  -0.439251   

symbol                   SOLUSDT   XLMUSDT   XRPUSDT  
fast_window low_window                                
10          20         -0.625314 -0.296199 -0.536097  
            30         -0.635065 -0.595105 -0.501568  
            40         -0.563402 -0.640572 -0.479396  
            50         -0.324340 -0.524803 -0.355799  
            60         -0.461152 -0.593942 -0.477941  
...                          ...       ...       ...  
210         230        -0.030279 -0.313776 -0.182179  
            240        -0.213148 -0.331099 -0.041941  
220         230        -0.163262 -0.291206 -0.155728  
            240        -0.330243 -0.381163 -0.220935  
230         240        -0.498400 -0.496739 -0.120590  

[276 rows x 18 columns]

In [8]:
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Max total return', height=400, width=1700)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

#### Testing RSI

In [9]:
# rsi = vbt.RSI.run(data)
# exits = rsi.rsi_crossed_above([60, 70, 80])
# entries = rsi.rsi_crossed_below([20, 30, 40])

grid = np.arange(20, 90, 5)
rsi = vbt.RSI.run(data, short_name='RSI')
rsi_top, rsi_bottom = vbt.RSI.run_combs(
    data,
    window=grid,
    short_names=['RSI_top', 'RSI_bottom']
)
entries = rsi.rsi_crossed_below(rsi_bottom)
exits = rsi.rsi_crossed_above(rsi_top)

In [10]:
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
comb = pf.total_return().unstack('symbol')
comb

symbol                             ADAUSDT  ALGOUSDT  ATOMUSDT   BCHUSDT  \
RSI_bottom_window RSI_top_window                                           
25                20             -0.787866 -0.811038 -0.744598 -0.769376   
30                20             -0.749547 -0.832616 -0.654002 -0.717328   
                  25             -0.824230 -0.816491 -0.712994 -0.727929   
35                20             -0.756860 -0.747975 -0.706727 -0.704314   
                  25             -0.828025 -0.803223 -0.811492 -0.636378   
...                                    ...       ...       ...       ...   
85                60             -0.834984 -0.787040 -0.820735 -0.593474   
                  65             -0.821375 -0.771710 -0.855986 -0.611687   
                  70             -0.853120 -0.778996 -0.838267 -0.637964   
                  75             -0.823835 -0.735336 -0.823440 -0.723618   
                  80             -0.801996 -0.747056 -0.826172 -0.642019   

symbol                             BNBUSDT   BTCUSDT  DOGEUSDT   DOTUSDT  \
RSI_bottom_window RSI_top_window                                           
25                20             -0.682372 -0.746121 -0.809075 -0.688220   
30                20             -0.725779 -0.736899 -0.843899 -0.716340   
                  25             -0.774066 -0.697049 -0.785760 -0.838788   
35                20             -0.767905 -0.701537 -0.850525 -0.670828   
                  25             -0.799252 -0.714931 -0.829301 -0.787205   
...                                    ...       ...       ...       ...   
85                60             -0.768363 -0.599014 -0.784251 -0.809016   
                  65             -0.781104 -0.625946 -0.763573 -0.818024   
                  70             -0.771158 -0.676113 -0.754880 -0.786598   
                  75             -0.803811 -0.687515 -0.806008 -0.790792   
                  80             -0.795358 -0.740339 -0.844128 -0.775181   

symbol                             ETCUSDT   ETHUSDT   FILUSDT  LINKUSDT  \
RSI_bottom_window RSI_top_window                                           
25                20             -0.544459 -0.824465 -0.808146 -0.868915   
30                20             -0.661735 -0.781708 -0.837276 -0.886007   
                  25             -0.784256 -0.734097 -0.861361 -0.879854   
35                20             -0.651114 -0.742396 -0.797103 -0.867134   
                  25             -0.756976 -0.734394 -0.818654 -0.870561   
...                                    ...       ...       ...       ...   
85                60             -0.663038 -0.755548 -0.844553 -0.819178   
                  65             -0.734633 -0.800444 -0.853622 -0.777452   
                  70             -0.673678 -0.817385 -0.857469 -0.820419   
                  75             -0.710278 -0.793935 -0.813394 -0.879745   
                  80             -0.686330 -0.769870 -0.839568 -0.844557   

symbol                             LTCUSDT  LUNAUSDT  MATICUSDT   SOLUSDT  \
RSI_bottom_window RSI_top_window                                            
25                20             -0.795362 -0.348122  -0.648538 -0.790639   
30                20             -0.745529 -0.327029  -0.701806 -0.843617   
                  25             -0.660027 -0.190211  -0.723439 -0.716146   
35                20             -0.750278  0.019078  -0.445938 -0.846191   
                  25             -0.639834  0.020631  -0.558271 -0.767715   
...                                    ...       ...        ...       ...   
85                60             -0.728119 -0.528738  -0.880102 -0.762334   
                  65             -0.767093 -0.541522  -0.899974 -0.793431   
                  70             -0.806524 -0.533745  -0.848975 -0.808384   
                  75             -0.782307 -0.575054  -0.855847 -0.747223   
                  80             -0.788540 -0.584603  -0.829705 -0.815168   

symbol                         

In [11]:
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Max total return', height=400, width=1700)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

### Price Dinamics

In [12]:
df = dataset.groupby('symbol')['close']\
    .resample('30Min')\
    .last()\
    .loc[['LUNAUSDT']]\
    .unstack('symbol')

df.vbt.plot(height=400, width=1700)

FigureWidget({
    'data': [{'name': 'LUNAUSDT',
              'showlegend': True,
              'type': 'scat…

### RSI Strategy

In [13]:
rsi = vbt.RSI.run(df['LUNAUSDT'])
fig = vbt.make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    shared_yaxes=False,
    vertical_spacing=0.05,
)
df.vbt['LUNAUSDT'].plot(trace_kwargs=dict(name='Close'), fig=fig, add_trace_kwargs=dict(row=1, col=1))
rsi.rsi.vbt.plot(fig=fig, trace_kwargs=dict(name='RSI'), height=400, width=1700, add_trace_kwargs=dict(row=2, col=1))

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…

In [14]:
rsi = vbt.RSI.run(df['LUNAUSDT'])
entries = rsi.rsi_crossed_above(45)
exits = rsi.rsi_crossed_below(25)
pf = vbt.Portfolio.from_signals(df['LUNAUSDT'], entries, exits, fees=0.001)
pf.stats()

Start                               2021-11-01 00:00:00
End                                 2022-04-30 11:00:00
Period                                180 days 11:30:00
Start Value                                       100.0
End Value                                    214.068753
Total Return [%]                             114.068753
Benchmark Return [%]                          95.315399
Max Gross Exposure [%]                            100.0
Total Fees Paid                               38.315734
Max Drawdown [%]                              35.732859
Max Drawdown Duration                  62 days 17:00:00
Total Trades                                        108
Total Closed Trades                                 108
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  35.185185
Best Trade [%]                                50.992368
Worst Trade [%]                              -12

In [15]:
fast_ma = vbt.MA.run(df, 180, short_name='fast')
slow_ma = vbt.MA.run(df, 240, short_name='slow')

fig = df.vbt['LUNAUSDT'].plot(trace_kwargs=dict(name='Close'))
fast_ma.ma.vbt.plot(fig=fig, trace_kwargs=dict(name='fast MA'), height=400, width=1700)
slow_ma.ma.vbt.plot(fig=fig, trace_kwargs=dict(name='slow MA'), height=400, width=1700)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…

### Mooving Average Strategy

In [16]:
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(df['LUNAUSDT'], entries, exits, fees=0.001)
pf.stats()

/tmp/ipykernel_39270/3807709821.py:5: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7fa3dc20c940>. Pass column to select a single column/group.



Start                               2021-11-01 00:00:00
End                                 2022-04-30 11:00:00
Period                                180 days 11:30:00
Start Value                                       100.0
End Value                                    294.219946
Total Return [%]                             194.219946
Benchmark Return [%]                          95.315399
Max Gross Exposure [%]                            100.0
Total Fees Paid                                8.976202
Max Drawdown [%]                              28.462488
Max Drawdown Duration                  60 days 20:30:00
Total Trades                                       22.0
Total Closed Trades                                22.0
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                       50.0
Best Trade [%]                                71.714908
Worst Trade [%]                               -1

## Combination strategies using Moving Average Cross Strategy & RSI Strategy

In [156]:

def get_signals(data, fast_window, slow_window, rsi_window):
    rsi = vbt.RSI.run(data, window=rsi_window, short_name='RSI').rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window, short_name='fast').ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window, short_name='slow').ma.to_numpy()
    entries = (fast > slow) & (rsi < 25)
    exits = (fast < slow) & (rsi > 45)
    return entries, exits

entries, exits = get_signals(data, 180, 240, 14)
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Max total return', height=400, width=1700)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

In [17]:

def combine_rsi_ma(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(data, window=rsi_window).rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window).ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window).ma.to_numpy()
    indicator = np.where((fast > slow) & (rsi < top), 1, 0)
    indicator = np.where((fast < slow) & (rsi > bottom), -1, indicator)
    return indicator


indicator_rsi_ma = vbt.IndicatorFactory(
    class_name='Combination_RSI_MA',
    short_name='RSI_MA',
    input_names=['close'],
    param_names=['fast_window', 'slow_window', 'rsi_window', 'top', 'bottom'],
    output_names=['value'],
).from_apply_func(combine_rsi_ma, fast_window=180, slow_window=240, rsi_window=14, top=70, bottom=30)


res = indicator_rsi_ma.run(
    data,
    fast_window=180,
    slow_window=240,
    rsi_window=14,
    top=80,
    bottom=20
)


entries = res.value_crossed_above(0)
exits = res.value_crossed_below(0)
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Max total return', height=400, width=1700)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

### Multi Timeframes trading

In [18]:

close = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('5Min')\
    .last()\
    .unstack()\
    .T['BTCUSDT']

close.columns = ['close']
btc_5min = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('10Min')\
    .last()\
    .unstack()\
    .T['BTCUSDT']
btc_5min.columns = ['close']

btc_1hour = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('1H')\
    .last()\
    .unstack()\
    .T['BTCUSDT']
btc_1hour.columns = ['close']

def trade_multi_timeframe(close, data_1, data_2, rsi_window_1, rsi_window_2):

    rsi_5m = vbt.RSI.run(data_1, window=rsi_window_1, short_name='RSI').rsi
    rsi_5m, _ = rsi_5m.align(close, broadcast_axis=0, method='ffill', join='left')

    rsi_1h = vbt.RSI.run(data_2, window=rsi_window_2, short_name='RSI').rsi
    rsi_1h, _ = rsi_1h.align(close, broadcast_axis=0, method='ffill', join='left')

    signal = np.where((rsi_5m < 30) and (rsi_1h < 30), 1, 0)
    signal = np.where((rsi_5m > 70) and (rsi_1h > 70), -1, signal)
    return signal


indicator_multi_timeframe = vbt.IndicatorFactory(
    class_name='Combination_Multi_Timeframe',
    short_name='Multi_Timeframe',
    input_names=['close'],
    param_names=['data_1', 'data_2', 'rsi_window_1', 'rsi_window_2'],
    output_names=['pandas'],
).from_apply_func(trade_multi_timeframe)


# res = indicator_multi_timeframe.run(
#     close=close,
#     data_1=btc_5min,
#     data_2=btc_1hour,
#     rsi_window_1=14,
#     rsi_window_2=14
# )


# entries = res.value_crossed_above(0)
# exits = res.value_crossed_below(0)
# pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
# max_return = pf.total_return().groupby('symbol').max()
# max_return.vbt.barplot(xaxis_title='Symbol',
#                        yaxis_title='Max total return', height=400, width=1700)


In [19]:

close = dataset.copy()\
    .groupby('symbol')['close']\
    .resample('5Min')\
    .last()\
    .unstack()\
    .T

close_15m = dataset.copy().\
    groupby('symbol')['close']\
    .resample('15Min')\
    .last()\
    .unstack()\
    .T
close_1h = dataset.copy().\
    groupby('symbol')['close']\
    .resample('60Min')\
    .last()\
    .unstack()\
    .T

rsi_15m = vbt.RSI.run(close_15m, window=14, short_name='RSI').rsi
rsi_15m, _ = rsi_15m.align(close, broadcast_axis=0,
                           method='ffill', join='right')

rsi_1h = vbt.RSI.run(close_1h, window=14, short_name='RSI').rsi
rsi_1h, _ = rsi_1h.align(close, broadcast_axis=0, method='ffill', join='right')

signal = rsi_15m.copy().applymap(lambda x: 0)
signal[(rsi_15m < 20 ) & (rsi_1h < 30)] = 1
signal[(rsi_15m > 50 ) & (rsi_1h > 70)] = -1

entries = np.where(signal == 1, True, False)
exits = np.where(signal == -1, True, False)

pf = vbt.Portfolio.from_signals(close, entries, exits, fees=0.001)
max_return = pf.total_return().groupby('symbol').max()
max_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Max total return', height=400, width=1700)


FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…

In [154]:
momo_bands_sma_ta = [
    {"kind":"sma", "length": 50},
    {"kind":"sma", "length": 200},
    {"kind":"bbands", "length": 20, "ddof": 0},
    {"kind":"macd"},
    {"kind":"rsi"},
    {"kind":"log_return", "cumulative": True}
]
momo_bands_sma_strategy = ta.Strategy(
    "Momo, Bands and SMAs and Cumulative Log Returns", # name
    momo_bands_sma_ta, # ta
    "MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns" # description
)
momo_bands_sma_strategy

Strategy(name='Momo, Bands and SMAs and Cumulative Log Returns', ta=[{'kind': 'sma', 'length': 50}, {'kind': 'sma', 'length': 200}, {'kind': 'bbands', 'length': 20, 'ddof': 0}, {'kind': 'macd'}, {'kind': 'rsi'}, {'kind': 'log_return', 'cumulative': True}], description='MACD and RSI Momo with BBANDS and SMAs 50 & 200 and Cumulative Log Returns', created='Thursday May 5, 2022, NYSE: 10:13:18, Local: 14:13:18 EEST, Day 125/365 (34.00%)')

In [155]:
df_st = df['LUNAUSDT'].copy()
df_st = pd.DataFrame(df_st)
df_st.columns = ['close']
df_st.ta.strategy(momo_bands_sma_strategy)
df_st

,close,SMA_50,SMA_200,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,CUMLOGRET_1
time,,,,,,,,,,,,,
2021-11-01 00:00:00,43.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2021-11-01 00:30:00,43.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002085
2021-11-01 01:00:00,43.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000927
2021-11-01 01:30:00,43.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001857
2021-11-01 02:00:00,42.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30 09:00:00,83.61,85.6882,88.79785,84.171985,85.2745,86.377015,2.585802,-0.254865,-0.336270,-0.126613,-0.209657,30.874444,0.662176
2022-04-30 09:30:00,83.87,85.6330,88.73895,83.968437,85.2225,86.476563,2.943034,-0.039247,-0.395859,-0.148961,-0.246898,34.746154,0.665281
2022-04-30 10:00:00,83.65,85.5764,88.68055,83.710092,85.1265,86.542908,3.327772,-0.021213,-0.455584,-0.166949,-0.288635,33.058877,0.662655
